In [ ]:
import pandas as pd

from Bio import PDB

# 3->1-betűs aminosav-kód konverzió dictionary
from Bio.Data.IUPACData import protein_letters_3to1

In [ ]:
# PDB beolvasása, residues a residuek listája
parser = PDB.PDBParser()
structure = parser.get_structure("2cr4_A_SH2", "2cr4_A_SH2.pdb") # Sok-modelles pdb-fájlokat nem szereti!
residues = PDB.Selection.unfold_entities(structure, "R")

In [ ]:
# az aktuális chain ID egyszerűen kiszedhető a fájlnévből

chain="2cr4_A_SH2.pdb".split('_')[1]
chain

In [ ]:
# tuple-ök listájaként kiszedjük a residue-kat egybetűs kóddal és számmal
# egybetűs kód: i.get_resname(), ezt átalakítjuk titlecase-re: .title(), és lookup-oljuk a dictionaryből:
# BSI.protein_letters_3to1[]
# residue szám: i.get_id()[1], az 1-es index azért kell, mert nézd meg mit ad anélkül :)

pdbseq=list(zip([BSI.protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))

In [ ]:
print([i for i in pdbseq])

In [ ]:
# Beolvassuk a táblázatot (itt csak egy példa) -> ennek a táblázatnak nem találtam meg a legújabb verzióját: 
# amiből ez a példa van, abban nincsenek a számok korrigálva (ez nagyon fontos!), a multiindex sincs bevezetve 
# és a PDB ID-k is rossz oszlopban vannak, de ez utóbbi mondjuk nem is kell, ha a lentebbi megoldást ciklusba foglalod és
# azzal töltöd fel a táblázatot
table=pd.read_csv('table_pelda.csv',index_col=[0,1])

In [ ]:
table

In [ ]:
# Beolvassuk az aktualizált pfam táblázatot
pfam=pd.read_csv('SH2_domain_containing_prot_right_resnum.csv')

In [ ]:
# Kiszámoljuk a számozások közötti különbséget a STOP pozíciók alapján
increment=int(pfam[pfam['PDB ID']=='2CR4']['New_uniprot_stop']-pfam[pfam['PDB ID']=='2CR4']['New_PDB_stop'])
increment

# ez a legtöbb esetben működik, mert a starttal ellentétben itt nincsenek nullák.
# Kivétel: 619-620-as sorok (TNS2), illetve egyéb hibák is vannak, pl. 603-as sor STAP1, az előző sor (STAT5B)
# számai "átlógnak ide", az ilyen hibákat javítani kellene, de egyelőre nem ez a prioritás.

In [ ]:
# Hozzáadunk egy sort, amit - karakterekkel töltünk ki alapból
# Figyeld meg a multiindexelést: tuple, aminek az első eleme a fehérjenév

table.loc[('SH3BP2','2CR4_'+chain),:] = '-'

# a chain ID másképp is tárolható persze

In [ ]:
table

In [ ]:
# Most pedig adjuk hozzá egyenként az egykódos aminosavakat a megfelelő cellákhoz. Ehhez az első trükk: szedjük ki
# az adott (Uniprot) pozícióhoz tartozó oszlopindexet! Ehhez megnézzük hogy az adott fehérje "numbers" sorában melyik
# oszlopindexnél van az adott szám, ez a megoldás stackoverflow-ról van: https://stackoverflow.com/questions/52173161/getting-a-list-of-indices-where-pandas-boolean-series-is-true

# (sztringgé kell alakítani előbb)

s=table.loc[('SH3BP2','numbers')]==str(434)
s[s].index[0]

In [ ]:
# Az előzőt végigiteráljuk a PDB minden egyes residue-jára, úgy hogy a PDB számozáshoz hozzáadjuk az "increment"-et

for (resname,resid) in pdbseq:
    s=table.loc[('SH3BP2','numbers')]==str(resid+increment)
    col=s[s].index[0]
    #eddig ugyanaz mint fentebb, csak az oszlopindexet mentjük a col változóba
    
    table.loc[('SH3BP2','2CR4_A'),col]=resname
    # ezzel pedig beírtuk resname-et a megfelelő helyre

In [ ]:
# És kész! Az eredményt lásd itt:

table.to_csv('table_pelda_out.csv')

# Figyeld meg hogy az aminosavak el vannak csúszva eggyel a kanonikus szekvenciához képest! Ezért fontos a számok
# javítása a táblázatban! (+1 az összeshez, illetve a sor végén még van +4 pozíció!)